<a href="https://colab.research.google.com/github/johncoogan53/XAI_HW6/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AIPI 590 - XAI | Assignment 06
Description:

This assignment will utilize

John Coogan

In [ ]:
def placeholder():
    return 0

placeholder()